In [ ]:
import os

filename="outputs/ComfyUI_00309_.png"

file=os.path.basename(filename)
file

In [ ]:
from llama_cpp import Llama
import json

model_id="/Users/janghyeonbin/ai-companion/models/llm/gguf/Meta-Llama-3.1-8B-Instruct.Q4_K_M.gguf"
model=Llama(model_id)

chat="안녕?"
response = model.create_chat_completion(
    messages=[
        {"role": "system", "content": "당신은 유용한 AI 비서입니다."},
        {"role": "user", "content": chat}
    ]
)

response["choices"][0]["message"]["content"]

In [ ]:
from diffusers import AutoPipelineForText2Image


In [ ]:
pipeline = AutoPipelineForText2Image.from_pretrained("runwayml/stable-diffusion-v1-5")

In [ ]:
from huggingface_hub import snapshot_download

In [ ]:
snapshot_download(repo_id="faridlazuarda/valadapt-llama-3.1-8B-it-korean", local_dir="./models/llm/loras/valadapt-llama-3.1-8B-it-korean")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path="/Volumes/EXDATA/models/LLM/transformers/llama/meta-llama__Llama-3.1-8B-Instruct"
lora_path="/Users/janghyeonbin/ai-companion/models/llm/loras/valadapt-llama-3.1-8B-it-korean"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path, device_map="auto", torch_dtype="auto")
model.load_adapter(lora_path)

In [ ]:
import cv2
import imutils
import pytesseract

In [ ]:
image=cv2.imread('./outputs/ComfyUI_00318_.png')

In [ ]:
image.shape

In [ ]:
from PIL import Image, ImageOps

In [ ]:
original_image=Image.open('ComfyUI_00318_.png').convert('RGBA')
original_image.show()

In [ ]:
image=Image.open('layer_0.png').convert('RGBA')
image.show()

In [ ]:
new_alpha=image.getchannel('A')
new_alpha.show()

In [ ]:
new_alpha=ImageOps.invert(new_alpha)
new_alpha.show()

In [ ]:
new_mask=Image.new("RGBA", image.size)
new_mask.show()

In [ ]:
new_mask.putalpha(new_alpha)
new_mask.show()

In [ ]:
from huggingface_hub import HfApi

In [ ]:
from dotenv import load_dotenv

In [ ]:
load_dotenv()

In [ ]:
import os

In [ ]:
token=os.getenv('HF_TOKEN')
token

In [ ]:
api=HfApi(token=token)

In [ ]:
models=api.list_models()

In [ ]:
models=api.list_models(task='audio-text-to-text', library="transformers")

In [ ]:
list(models)

In [ ]:
from gtts import gTTS
from pydub import AudioSegment
import matplotlib.pyplot as plt
import scipy.fftpack
import scipy.io.wavfile
import numpy as np

In [ ]:
def text2mp3():
    tts=gTTS('Hello',lang='en')
    tts.save('text1.mp3')

In [ ]:
text2mp3()

In [ ]:
def text2wav():
    tts=gTTS('Hello', lang="en")
    tts.save('hello.mp3')
    
    w=AudioSegment.from_mp3('hello.mp3')
    w.export('Hello.wav', format='wav')

In [ ]:
text2wav()

In [ ]:
def text2wav(text, lang):
    tts=gTTS(text, lang=lang)
    tts.save(f"{text}.mp3")
    
    w=AudioSegment.from_mp3(f"{text}.mp3")
    w.export(f"{text}.wav", format="wav")

In [ ]:
text2wav('Hello', lang='en')

In [ ]:
text2wav('안녕하세요', lang='ko')

In [ ]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader

loader = DirectoryLoader(path='characters_info', glob="*.txt", loader_cls=TextLoader)
docs = loader.load()
docs.sort(key=lambda x: x.metadata.get('source', ''))

len(docs)

In [ ]:
docs

In [ ]:
import os
import re
from langchain.docstore.document import Document

In [ ]:
def split_document_by_sections(text):
    """
    텍스트에서 [항목] 형태의 섹션을 찾아서 (섹션 제목, 내용) 튜플 리스트로 반환하는 함수.
    """
    # 정규표현식 패턴: 대괄호 안에 항목명, 그 다음 내용은 다음 대괄호가 나오기 전까지
    pattern = re.compile(r'\[(.*?)\]\s*(.*?)(?=\n\s*\[|$)', re.DOTALL)
    sections = []
    for match in pattern.finditer(text):
        section_title = match.group(1).strip()
        section_content = match.group(2).strip()
        sections.append((section_title, section_content))
    return sections

In [ ]:
# 언어 이름과 코드 매핑 딕셔너리
LANGUAGE_MAP = {
    "한국어": "ko",
    "日本語": "ja",
    "简体中文": "zh_CN",
    "繁體中文": "zh_TW",
    "English": "en"
}

def extract_language_code(heading):
    """
    헤딩 텍스트 (예: "## 한국어")에서 언어 이름을 추출하여
    매핑 딕셔너리를 통해 언어 코드를 반환하는 함수.
    """
    match = re.match(r"##\s*(.+)", heading)
    if match:
        lang_name = match.group(1).strip()
        return LANGUAGE_MAP.get(lang_name, lang_name)
    return None


In [ ]:
def process_multilang_document(doc):
    """
    파일 하나에 여러 언어 섹션이 있을 때,
    각 언어 헤딩(예: "## 한국어")을 기준으로 블록을 분리하고,
    각 블록 내에서 [설정], [성격] 등 섹션별로 분리한 Document 리스트를 반환.
    """
    text = doc.page_content
    # 언어 헤딩(## 로 시작하는 줄)을 기준으로 분리 (헤딩을 포함하도록 분리)
    blocks = re.split(r'(?=##\s*)', text)
    
    processed = []
    for block in blocks:
        if block.startswith("##"):
            # 첫 줄에서 언어 헤딩을 추출
            lines = block.splitlines()
            lang_code = extract_language_code(lines[0])
            # 헤딩 부분을 제거한 나머지 내용 사용
            content = "\n".join(lines[1:]).strip()
            # 이전에 정의한 섹션 분리 함수 사용
            sections = split_document_by_sections(content)
            for section_title, section_content in sections:
                new_doc = Document(
                    page_content=section_content,
                    metadata={
                        "language": lang_code,
                        "section": section_title,
                        "source": doc.metadata.get("source", "unknown")
                    }
                )
                processed.append(new_doc)
        else:
            # 만약 블록이 언어 헤딩 없이 나온다면, 무시하거나 기본값 설정 가능
            pass
    return processed

In [ ]:
all_processed_docs = []

In [ ]:
for doc in docs:
    print(doc)
    processed_docs = process_multilang_document(doc)
    all_processed_docs.extend(processed_docs)


In [ ]:
len(all_processed_docs)

In [ ]:
for doc in all_processed_docs:
    print(doc.metadata, doc)

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS, Chroma
from langchain_community.vectorstores.utils import DistanceStrategy

In [ ]:
embedding_model_name = "BAAI/bge-m3"

In [ ]:
embedding_model=HuggingFaceEmbeddings(
    model_name=embedding_model_name,
    encode_kwargs={"normalize_embeddings": True},
)

In [ ]:
vectorstore=FAISS.from_documents(
    documents=all_processed_docs, embedding=embedding_model, distance_strategy=DistanceStrategy.EUCLIDEAN_DISTANCE
)

In [ ]:
vector_count = vectorstore.index.ntotal

In [ ]:
vector_count

In [ ]:
user_query = "미나미 아스카의 성격과 외모에 대해 어떻게 생각해?"

In [ ]:
query_vector = embedding_model.embed_query(user_query)

In [ ]:
len(query_vector)

In [ ]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
basic_docs = retriever.invoke(user_query)

In [ ]:
for doc in basic_docs:
    print(doc.metadata, doc)

In [ ]:
from transformers import AutoTokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')

In [ ]:
tokenizer.chat_template

In [ ]:
from transformers import AutoConfig

In [ ]:
config = AutoConfig.from_pretrained('meta-llama/Meta-Llama-3.1-8B-Instruct')

In [ ]:
config.top_k

In [ ]:
config.top_p

In [ ]:
config.eos_token_id

In [ ]:
config.pad_token_id

In [ ]:
config.temperature

In [ ]:
config.repetition_penalty

In [ ]:
import gradio as gr

gr.themes.builder()

In [ ]:
import os
from pathlib import Path
import torch
from safetensors import safe_open
from safetensors.torch import save_file
from tqdm.auto import tqdm

# 変換するディレクトリを指定します
input_dir = "/tmp/ComfyUI/models/checkpoints/"

# 指定したディレクトリ内のすべてのsafetensorsファイルを取得
safetensors_files = list(Path(input_dir).glob("*.safetensors"))

def convert_model(file_path: str):
    print(f"変換中: {file_path}")
    
    # メタデータの読み込み
    try:
        with safe_open(file_path, framework="pt", device="cpu") as f:
            metadata = f.metadata()
            metadata = metadata if metadata is not None else {}
    except Exception as e:
        print(f"メタデータの読み取り中にエラーが発生しました: {str(e)}")
        return False

    # モデルの変換
    try:
        sd_pruned = {}
        with safe_open(file_path, framework="pt", device="cpu") as f:
            for key in tqdm(f.keys(), desc="テンソルを変換中"):
                tensor = f.get_tensor(key)
                sd_pruned[key] = tensor.to(torch.float8_e4m3fn)

        # 変換したモデルを保存（元のディレクトリに）
        output_dir = os.path.dirname(file_path)
        model_name = os.path.splitext(os.path.basename(file_path))[0]
        output_filename = f"{model_name}_fp8.safetensors"
        output_path = os.path.join(output_dir, output_filename)
        save_file(sd_pruned, output_path, metadata={"format": "pt", **metadata})
        print(f"ファイルが正常に保存されました: {output_path}")

        return True
    except Exception as e:
        print(f"変換中にエラーが発生しました: {str(e)}")
        return False
        

In [ ]:
file_path = "models/diffusion/checkpoints/sdxl/animagine-xl-4.0-opt.safetensors"

In [ ]:
convert_model(file_path)

In [ ]:
def convert_model_to_int8(file_path: str):
    print(f"Converting: {file_path}")
    
    # Read metadata
    try:
        with safe_open(file_path, framework="pt", device="cpu") as f:
            metadata = f.metadata()
            metadata = metadata if metadata is not None else {}
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return False

    # Convert the model
    try:
        sd_pruned = {}
        with safe_open(file_path, framework="pt", device="cpu") as f:
            for key in tqdm(f.keys(), desc="Converting tensors"):
                tensor = f.get_tensor(key)
                sd_pruned[key] = tensor.to(torch.float8_e4m3fn)

        # Save the converted model (in the original directory)
        output_dir = os.path.dirname(file_path)
        model_name = os.path.splitext(os.path.basename(file_path))[0]
        output_filename = f"{model_name}_fp8.safetensors"
        output_path = os.path.join(output_dir, output_filename)
        save_file(sd_pruned, output_path, metadata={"format": "pt", **metadata})
        print(f"Saved: {output_path}")

        return True
    except Exception as e:
        print(f"ERROR: {str(e)}")
        return False
        